In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,random
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

running_dir='/kaggle/input'

def try_cmd(cmd):
    return os.popen(cmd).read()

def findall_in_list(self, content):
    """
    从列表中找出所有包含content的元素
    :param self: 列表本身
    :param content: 查找内容
    :return res_li: list 包含content的列表
    """
    try:
        content = str(content)
    except Exception as err:
        print('string failed !', err)
    res_li = []
    for x in self:
        try:
            if content in str(x):
                res_li.append(x)
        finally:
            pass
    return res_li


# list.findall_in_list = findall_in_list  # TypeError: can't set attributes of built-in/extension type 'list'


def findall_in_dir(content, path=running_dir, include_dir=True) -> list:
    """

    :param content: 查找内容
    :param path: 查找的根文件夹
    :param include_dir: 是否返回包含纯文件夹的结果
    :return res_li: list 包含content的列表:
    """
    res_li = []
    for ro, dirs, files in os.walk(path):
        if include_dir:
            res_li += findall_in_list([ro + os.sep + d for d in dirs], content)
        res_li += findall_in_list([ro + os.sep + f for f in files], content)
    return res_li


def content_walker(content, path=running_dir, include_dir=True):
    """

    :param content: 查找内容
    :param path: 查找的根文件夹
    :param include_dir: 是否返回包含纯文件夹的结果
    :return : generator 和 os.walk 输出模式一致
    """
    if include_dir:
        for ro, dirs, files in os.walk(path):
            if content in ro:
                yield ro, dirs, files
            else:
                dirs_with_content = findall_in_list([d for d in dirs], content)
                files_with_content = findall_in_list([f for f in files], content)
                if dirs_with_content + files_with_content:
                    yield ro, dirs_with_content, files_with_content
    else:
        for ro, dirs, files in os.walk(path):
            if content in ro:
                yield ro, files
            else:
                files_with_content = findall_in_list([f for f in files], content)
                if files_with_content:
                    yield ro, files_with_content

def get_id(url_):
    return url_.split('/')[-1].split('.')[0]
                    

def process_c(url_, **kargs):
    data_x = np.load(url_)
    id_ = get_id(url_)
    img_b = np.vstack([data_x[0, :, : ], data_x[2, :, : ], data_x[4, :, : ]])
    img_f = np.vstack([data_x[1, :, : ], data_x[3, :, : ], data_x[5, :, : ]])
    fig = plt.figure(figsize=(9, 12))
    ax = fig.add_subplot(2, 1, 1)
    ax.imshow(img_b.astype('float'), interpolation='nearest', aspect='auto')
    ax = fig.add_subplot(2, 1, 2)
    ax.imshow(img_f.astype('float'), interpolation='nearest', aspect='auto')
    ax.set_title(f'Id: {id_}, {kargs=}', size=16)
    return id_, [img_b, img_f]


def process_c1(url_, **kargs):
    data_x = np.load(url_)
    id_ = get_id(url_)
    img_dp1 = data_x[0, :, : ] - data_x[1, :, : ]
    img_dp2 = data_x[2, :, : ] - data_x[3, :, : ]
    img_dp3 = data_x[4, :, : ] - data_x[5, :, : ]
    fig = plt.figure(figsize=(9, 12))
    ax = fig.add_subplot(3, 1, 1)
    ax.imshow(img_dp1.astype('float'), interpolation='nearest', aspect='auto')
    ax = fig.add_subplot(3, 1, 2)
    ax.imshow(img_dp2.astype('float'), interpolation='nearest', aspect='auto')
    ax = fig.add_subplot(3, 1, 3)
    ax.imshow(img_dp3.astype('float'), interpolation='nearest', aspect='auto')
    ax.set_title(f'Id: {id_}, {kargs=}', size=16)
    return id_, [img_dp1, img_dp2, img_dp3]



# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

res_df = pd.read_csv(f'{running_dir}/seti-breakthrough-listen/sample_submission.csv').set_index('id')
# print(res_df)
train_df = pd.read_csv(f'{running_dir}/seti-breakthrough-listen/train_labels.csv').set_index('id')


train_npy_li = findall_in_dir('.npy', f'{running_dir}/seti-breakthrough-listen/train/')
for x in train_npy_li[:100]:
    id_ = get_id(x)
    target_ = train_df.loc[id_, 'target']
    if target_ > 0:
        id_, img_li = process_c(data_x, tar=target_)
        
    


    
test_npy_li = findall_in_dir('.npy', f'{running_dir}/seti-breakthrough-listen/test/')
for x in test_npy_li[:3]:
    id_, img_li = process_c(data_x, tar=target_)
    
    
    res = 0 if random.random() < 0.9369 else 1
    res_df.loc[id_, 'target'] = res
    
#     print(x, id_, )
#     print(res_df.loc[id_, :] )


# res_file_name=f'/kaggle/working/sbl-submission-03.csv'
# res_df.to_csv(res_file_name)

    
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df

In [ ]:
# try_cmd(f'kaggle competitions submit -c seti-breakthrough-listen -f {res_file_name} -m "try commit by kaggle nb"')